In [2]:
import requests
from typing import Optional, Dict, Any, Union

class BaseAPI:
    """BaseAPI serves as a wrapper for basic REST API operations with a given base URL.
    
    Attributes:
        base_url (str): The base URL for the API endpoints.
        headers (Dict[str, str]): Default headers to include in all requests.
    """

    def __init__(self, base_url: str = "http://localhost:8001/v1"):
        """Initialize the BaseAPI with a base URL and default headers.

        Args:
            base_url (str): The base URL for the API endpoints. Defaults to a sample ngrok URL.
        """
        self.base_url: str = base_url
        self.headers: Dict[str, str] = {"Accept": "application/json"}

    def _post(self, endpoint: str, data: Optional[Dict[str, Any]] = None, files: Optional[Dict[str, Any]] = None):# -> Optional[Dict[str, Any]]:
        """Sends a POST request to a specific endpoint.

        Args:
            endpoint (str): The API endpoint to post data to.
            data (Optional[Dict[str, Any]]): JSON data to send in the request. Defaults to None.
            files (Optional[Dict[str, Any]]): Files to upload. Defaults to None.

        Returns:
            Optional[Dict[str, Any]]: The JSON response from the API or None in case of an error.
        """
        url: str = f"{self.base_url}/{endpoint}"
        headers: Dict[str, str] = self.headers
        try:
            if files:
                response = requests.post(url, files=files, headers=headers)
            else:
                response = requests.post(url, json=data, headers=headers)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"Request Error: {e}")
        return None

    def _get(self, endpoint: str) -> Optional[Dict[str, Any]]:
        """Performs a GET request to a specified endpoint.

        Args:
            endpoint (str): The API endpoint to retrieve data from.

        Returns:
            Optional[Dict[str, Any]]: The JSON response from the API or None in case of an error.
        """
        url: str = f"{self.base_url}/{endpoint}"
        try:
            response = requests.get(url, headers=self.headers)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"Request Error: {e}")
        return None


In [3]:
from enum import Enum, auto
from typing import List, Dict, Optional, Any

class Role(Enum):
    USER = auto()
    SYSTEM = auto()
    ASSISTANT = auto()

    def __str__(self):
        return self.name.lower()

class ChatCompletionAPI(BaseAPI):
    """
    A specialized API client for handling chat completions with conversation history management,
    utilizing an enum for role management.
    """

    def __init__(self, base_url: str = "http://localhost:8001/v1"):
        super().__init__(base_url)
        self.conversation_history: List[Dict[str, Any]] = []


    def post_completions(self, messages: List[Dict[str, Any]], role: Optional[Role] = None, stream: bool = True, context_filter: Optional[str] = None, use_context: bool = True, include_source: bool = False) -> Optional[Dict[str, Any]]:
        """
        Post messages to the chat completions endpoint with optional parameters for role, streaming, and context configuration.

        Args:
            messages: A list of message dictionaries with "content" and optionally "role" keys.
            role: An optional role for all messages, overriding individual message roles if provided. Defaults to None.
            stream: Whether to enable or disable streaming responses. Defaults to True.
            context_filter: Optional filter to apply to the context, affecting which parts are used. Defaults to None.
            use_context: Whether to use the existing context for generating completions. Defaults to True.
            include_source: Whether to include the source information in the response. Defaults to False.

        Returns:
            Optional[Dict[str, Any]]: The JSON response from the API or None in case of an error.
        """
        endpoint = "completions"
        payload = {
            "prompt": messages,
            "stream": stream,
            "context_filter": context_filter,
            "use_context": use_context,
            "include_source": include_source
        }
        
        # Add role to payload if it is specified
        if role is not None:
            payload["role"] = str(role)

        response = self._post(endpoint, payload)
        return response

    def post_chat_completions(self, message: str, role: Role = Role.USER, is_init: bool = False, stream: bool = False) -> Optional[Dict[str, Any]]:
        """
        Post a single message to the chat completions endpoint, managing conversation history.

        Args:
            message: The message content to send.
            role: The role of the message sender, represented as an Enum. Defaults to Role.USER.
            is_init: Whether to initialize (reset) the conversation history. Defaults to False.
            stream: Whether to enable streaming for this message. Defaults to False.

        Returns:
            The JSON response from the API or None in case of an error.
        """
        endpoint = "chat/completions"

        if is_init:
            self.conversation_history = []
            if not message:  # If initializing without a new message, return here.
                return None
            else:
                self.conversation_history.append({"content": message, "role": str(role)})
            return None

        else:
            # Add the new message with its role to the conversation history
            self.conversation_history.append({"content": message, "role": str(role)})

        payload = {
            "messages": self.conversation_history,
            "stream": stream
        }

        response = self._post(endpoint, payload)
        return response


In [4]:
class IngestAPI(BaseAPI):
    def ingest_file(self, file_path):
        """Ingest a file to a specific endpoint."""
        endpoint = "ingest/file"
        with open(file_path, 'rb') as f:
            files = {'file': (file_path.split('/')[-1], f)}
            response = self._post(endpoint, files=files)
        return response

    def ingest_text(self, file_name, text):
        """Ingest text with a file name."""
        endpoint = "ingest/text"
        payload = {"file_name": file_name, "text": text}
        return self._post(endpoint, data=payload)

    def list_ingest_jobs(self):
        """List all ingest jobs."""
        endpoint = "ingest/list"
        return self._get(endpoint)

In [5]:
class UtilAPI(BaseAPI):
    def get_embeddings(self, input_text):
        """Post data to the embeddings endpoint."""
        endpoint = "embeddings"
        payload = {"input": input_text}
        return self._post(endpoint, data=payload)

    def check_health(self):
        """Get the health status of the API."""
        endpoint = "health"
        return self._get(endpoint)
    
    def post_chunks(self, text):
        """Post chunks of text."""
        endpoint = "chunks"
        payload = {"text": text}
        return self._post(endpoint, data=payload)

## Chatting

In [17]:
chat_api = ChatCompletionAPI(base_url='https://0e80-161-28-242-150.ngrok-free.app/v1')

In [18]:
# Example usage of sending a single message
messages = "What is cs 2450 prerequisit?"

response = chat_api.post_completions(messages, stream=False)

message = response['choices'][0]['message']['content']
message

' The prerequisites for CS 2450 (Software Engineering) are CS 2300 and CS 2420.'

In [19]:
# Example usage of sending a single message with role management
messages = "What is cs 2450 prerequisit?"
response = chat_api.post_completions(messages, role=Role.USER, stream=False)

message = response['choices'][0]['message']['content']
message

' The prerequisites for CS 2450 (Software Engineering) are CS 2300 and CS 2420.'

In [20]:
# Example: Chatting with continuous messaging, check the implementation how the conversation history is managed with role management
stream_response = chat_api.post_chat_completions("You are UVU Advisor", role=Role.SYSTEM, is_init=True)
stream_response = chat_api.post_chat_completions("What is cs 2450 prerequisit?", role=Role.USER, is_init=False)

message = stream_response['choices'][0]['message']['content']
message

' CS 2450, which is "Operating Systems" at Utah Valley University (UVU), typically has the following prerequisites:\n\n1. A strong background in programming concepts and data structures using a high-level language such as C++ or Java. This would usually be fulfilled by taking CS 1420 ("Programming Concepts I") and CS 1520 ("Data Structures and Algorithms").\n2. Familiarity with the Linux operating system, including command line basics like file navigation, editing text files, compiling code, etc. This can be gained through taking a course such as CS 1310 ("Introduction to Linux") or having significant experience using Linux in other contexts.\n\nIt\'s important to note that these prerequisites may vary depending on the specific semester and instructor for the CS 2450 class, so it is always best to consult with your academic advisor or the department chair for confirmation before enrolling in the course.'

In [16]:
stream_response

{'id': '0d47e5ff-d107-4b65-8e2a-d4cfbe9d4e53',
 'object': 'completion',
 'created': 1711426291,
 'model': 'private-gpt',
 'choices': [{'finish_reason': 'stop',
   'delta': None,
   'message': {'role': 'assistant',
    'content': ' CS 2450, which is "Operating Systems" at Utah Valley University (UVU), typically has the following prerequisites:\n\n1. A strong background in programming concepts and data structures using a high-level language such as C++ or Java. This would usually be covered by taking courses like CS 1510 ("Programming Concepts I") and CS 2410 ("Data Structures and Algorithms").\n2. Familiarity with the Linux command line interface (CLI) is recommended but not always required, as some instructors may provide a Windows-based lab environment for students to use during class activities. However, having experience using the CLI can make learning the material in CS 2450 more efficient and effective.\n3. A solid foundation in computer organization and architecture concepts woul

## < Under Construction >

##  IngestAPI

In [ ]:
file_paths = [
    ('/Users/qratul/uvu/courses/swe/spring_24/project/base/uvu_advisor_bot_client/data/uvu_docs/course_catelog_cs_uvu.pdf', 'application/octet-stream'),
    ('/Users/qratul/uvu/courses/swe/spring_24/project/base/uvu_advisor_bot_client/data/uvu_docs/course_catelog_cs_uvu.txt', 'application/octet-stream')
]

files = [
    ('file', (file_path.split('/')[-1], open(file_path, 'rb'), content_type))
    for file_path, content_type in file_paths
]

response = requests.post(url, files=files)

# Correctly close the files after the request is made
for _, file_tuple in files:
    file_object = file_tuple[1]
    file_object.close()

print(response.text)

Making request to: https://4158-161-28-242-155.ngrok-free.app/v1/ingest/file with data: {'file': <_io.BufferedReader name='/Users/qratul/uvu/courses/swe/spring_24/project/base/uvu_advisor_bot_client/data/uvu_docs/course_catelog_cs_uvu.json'>}
Unexpected Error: Object of type BufferedReader is not JSON serializable
File Ingest Response: None


In [ ]:
# Initialize the utility API client
util_api = UtilAPI()

# Get embeddings
embeddings_response = util_api.get_embeddings("This is a test sentence.")
print("Embeddings Response:", embeddings_response)

# Check health
health_response = util_api.check_health()
print("Health Response:", health_response)

# Post chunks of text
chunks_response = util_api.post_chunks("This is a test sentence for chunking.")
print("Chunks Response:", chunks_response)


### Totally tesitng


In [ ]:

import requests

class GPTAPI:
    base_url = 'https://4158-161-28-242-155.ngrok-free.app/v1'

    def __init__(self, api_key):
        self.api_key = api_key

    def make_request(self, endpoint, data):
        headers = {
            'Authorization': f'Bearer {self.api_key}',
            'Content-Type': 'application/json'
        }
        response = requests.post(f'{self.base_url}/{endpoint}', json=data, headers=headers)
        return response.json()


In [ ]:
class CompletionAPI(GPTAPI):
    def __init__(self, api_key, streaming=False):
        super().__init__(api_key)
        self.endpoint = 'completions'
        self.streaming = streaming

    def complete(self, prompt, **kwargs):
        data = {'prompt': prompt, 'stream': self.streaming, **kwargs}
        return self.make_request(self.endpoint, data)

class ChatCompletionAPI(GPTAPI):
    def __init__(self, api_key, streaming=False):
        super().__init__(api_key)
        self.endpoint = 'chat/completions'
        self.streaming = streaming

    def chat(self, messages, **kwargs):
        data = {'messages': messages, 'stream': self.streaming, **kwargs}
        return self.make_request(self.endpoint, data)


In [ ]:
api_key = 'your_api_key_here'

# Example: Non-streaming completion
completion_api = CompletionAPI(api_key, streaming=False)
response = completion_api.complete(prompt="Hello, world!", use_context=False)
print(response)




{'id': 'da36ddce-4b3b-4e71-b178-089f6f627c75', 'object': 'completion', 'created': 1707644164, 'model': 'private-gpt', 'choices': [{'finish_reason': 'stop', 'delta': None, 'message': {'role': 'assistant', 'content': ' Hello there! It\'s nice to meet you. How can I help you today? If you have any questions or topics you\'d like me to write about, feel free to ask. I\'m here to provide information and answer questions to the best of my ability. Let me know if you have any specific topic in mind or if you have any general questions. I\'ll do my best to provide accurate and helpful responses. Let me know if you have any preferences for length or format of the response as well. I look forward to helping you out!\n\nHere is a simple "Hello, World!" program in various programming languages for your reference:\n\n* C:\n```c\n#include <stdio.h>\n\nint main() {\n  printf("Hello, World!\\n");\n  return 0;\n}\n```\n\n* Java:\n```java\npublic class HelloWorld {\n  public static void main(String[] ar

In [ ]:
# Example: Streaming chat completion
chat_api = ChatCompletionAPI(api_key, streaming=True)
response = chat_api.chat(messages=[{"role": "user", "content": "Hello!"}], use_context=False)
print(response)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
import requests

url = "https://4158-161-28-242-155.ngrok-free.app/v1/ingest/file"

# Replace these paths with the actual paths to your files
file_paths = [
    ('/Users/qratul/uvu/courses/swe/spring_24/project/base/uvu_advisor_bot_client/data/uvu_docs/course_catelog_cs_uvu.pdf', 'application/octet-stream'),
    ('/Users/qratul/uvu/courses/swe/spring_24/project/base/uvu_advisor_bot_client/data/uvu_docs/course_catelog_cs_uvu.txt', 'application/octet-stream')
]

files = [
    ('file', (file_path.split('/')[-1], open(file_path, 'rb'), content_type))
    for file_path, content_type in file_paths
]

response = requests.post(url, files=files)

# Correctly close the files after the request is made
for _, file_tuple in files:
    file_object = file_tuple[1]
    file_object.close()

print(response.text)


{"object":"list","model":"private-gpt","data":[{"object":"ingest.document","doc_id":"1e236604-9b28-4e24-8b70-a9da271dd229","doc_metadata":{"file_name":"course_catelog_cs_uvu.txt"}}]}


In [ ]:
import requests

class BaseAPI:
    def __init__(self, base_url="https://4158-161-28-242-155.ngrok-free.app/v1"):
        self.base_url = base_url
        self.headers = {"Accept": "application/json"}  # Removed "Content-Type" for flexibility

    def _post(self, endpoint, data=None, files=None):
        url = f"{self.base_url}/{endpoint}"
        headers = self.headers
        if files:
            # For file uploads, do not include JSON data and let requests handle content-type
            response = requests.post(url, files=files, headers=headers)
        else:
            # For JSON data, specify json parameter
            response = requests.post(url, json=data, headers=headers)
        try:
            response.raise_for_status()
            return response.json()
        except requests.exceptions.HTTPError as e:
            print(f"HTTP Error: {e.response.status_code} {e.response.text}")
        except Exception as e:
            print(f"Unexpected Error: {str(e)}")
        return None

    def _get(self, endpoint):
        url = f"{self.base_url}/{endpoint}"
        response = requests.get(url, headers=self.headers)
        return response.json()


In [ ]:
class IngestAPI(BaseAPI):
    def ingest_file(self, file_path):
        """Ingest a file to a specific endpoint."""
        endpoint = "ingest/file"
        with open(file_path, 'rb') as f:
            files = {'file': (file_path.split('/')[-1], f)}
            response = self._post(endpoint, files=files)
        return response

    def ingest_text(self, file_name, text):
        """Ingest text with a file name."""
        endpoint = "ingest/text"
        payload = {"file_name": file_name, "text": text}
        return self._post(endpoint, data=payload)

    def list_ingest_jobs(self):
        """List all ingest jobs."""
        endpoint = "ingest/list"
        return self._get(endpoint)


In [ ]:
# Assuming the classes have been defined as per the previous instructions

# Initialize the API with the base URL
ingest_api = IngestAPI(base_url="https://4158-161-28-242-155.ngrok-free.app/v1")

# Example: Ingest a file
file_path = "/Users/qratul/uvu/courses/swe/spring_24/project/base/uvu_advisor_bot_client/data/uvu_docs/course_catelog_cs_uvu.pdf"
file_ingest_response = ingest_api.ingest_file(file_path)
print(file_ingest_response)

# Example: Ingest text
file_name = "course_catelog_cs_uvu.pdf"
text_content = "This is an example text content."
text_ingest_response = ingest_api.ingest_text(file_name, text_content)
print(text_ingest_response)

# Example: List ingest jobs
list_jobs_response = ingest_api.list_ingest_jobs()
print(list_jobs_response)


Making request to: https://4158-161-28-242-155.ngrok-free.app/v1/ingest/file
{'object': 'list', 'model': 'private-gpt', 'data': [{'object': 'ingest.document', 'doc_id': '7c29f12a-1bfe-43e7-b382-5febe79fe3cb', 'doc_metadata': {'page_label': '1', 'file_name': '/Users/qratul/uvu/courses/swe/spring_24/project/base/uvu_advisor_bot_client/data/uvu_docs/course_catelog_cs_uvu.pdf'}}, {'object': 'ingest.document', 'doc_id': 'd47ea662-4fa2-4a25-bf33-652533f4817f', 'doc_metadata': {'page_label': '2', 'file_name': '/Users/qratul/uvu/courses/swe/spring_24/project/base/uvu_advisor_bot_client/data/uvu_docs/course_catelog_cs_uvu.pdf'}}, {'object': 'ingest.document', 'doc_id': '576607e7-eafd-44f5-8391-81bae0d58c85', 'doc_metadata': {'page_label': '3', 'file_name': '/Users/qratul/uvu/courses/swe/spring_24/project/base/uvu_advisor_bot_client/data/uvu_docs/course_catelog_cs_uvu.pdf'}}, {'object': 'ingest.document', 'doc_id': '3c9db033-c3d9-4102-9a9a-76c7a33ec84e', 'doc_metadata': {'page_label': '4', 'file

AttributeError: 'IngestAPI' object has no attribute 'ingest_text'

In [ ]:
class IngestAPI(BaseAPI):
    def ingest_file(self, file_path):
        """Ingest a file to a specific endpoint."""
        endpoint = "ingest/file"
        with open(file_path, 'rb') as f:
            files = {'file': (file_path.split('/')[-1], f)}
            response = self._post(endpoint, files=files)
        return response

    def ingest_text(self, file_name, text):
        """Ingest text with a file name."""
        endpoint = "ingest/text"
        payload = {"file_name": file_name, "text": text}
        return self._post(endpoint, data=payload)

    def list_ingest_jobs(self):
        """List all ingest jobs."""
        endpoint = "ingest/list"
        return self._get(endpoint)
    
    
ingest_api = IngestAPI(base_url="https://4158-161-28-242-155.ngrok-free.app/v1")   
file_name = "course_catelog_cs_uvu.pdf"
text_content = "This is an example text content."
text_ingest_response = ingest_api.ingest_text(file_name, text_content)
print(text_ingest_response)


Making request to: https://4158-161-28-242-155.ngrok-free.app/v1/ingest/text
HTTP Error: 500 Internal Server Error
None
